## Funciones para estrucuturar los datos de la vista

In [14]:
from pyrfc._cyrfc import Connection, ABAPApplicationError
import pandas as pd
import numpy as np
from typing import List
from datetime import datetime, timedelta
import win32com.client as win32
from pretty_html_table import build_table
from dotenv import load_dotenv
import os
from typing import Dict, Any
from sqlalchemy import create_engine

load_dotenv()

class SAPConnection:
    def __init__(self, ashost, sysnr, client, user, passwd, lang):
        """
        Inicializa la conexión a SAP.
        """
        self.connection_params = {
            'ashost': ashost,
            'sysnr': sysnr,
            'client': client,
            'user': user,
            'passwd': passwd,
            'lang': lang
        }
        self.connection = None

    def open_connection(self):
        """
        Abre una conexión a SAP con los parámetros especificados.
        """
        if self.connection is None:
            self.connection = Connection(**self.connection_params)
        return self.connection

    def close_connection(self):
        """
        Cierra la conexión a SAP.
        """
        if self.connection:
            self.connection.close()
            self.connection = None

    def execute_query(self, query_name, view_id, parameters):
        """
        Ejecuta una consulta a SAP con parámetros dinámicos.

        Args:
            query_name (str): Nombre del query SAP.
            view_id (str): ID de la vista.
            parameters (list of tuples): Lista de parámetros en formato (nombre, valor).

        Returns:
            dict: Resultado de la consulta.
        """
        self.open_connection()
        # Construir la lista de parámetros dinámicamente
        formatted_parameters = [{"NAME": p[0], "VALUE": p[1]} for p in parameters]
        formatted_parameters += [
        ]

        print("🛠 Sending Parameters to SAP BW:") ##DEBUGGING
        for param in formatted_parameters:
            print(f"{param['NAME']} = {param['VALUE']}")


        try: 
            result = self.connection.call(
                "RRW3_GET_QUERY_VIEW_DATA",
                I_QUERY=query_name,
                I_VIEW_ID=view_id,
                I_T_PARAMETER=formatted_parameters
            )
            print("✅ Query executed successfully!") ##DEBUGGING
            return result
        except ABAPApplicationError as error:
            print("Error en SAP: " + error.message)
            return None
        finally:
            self.close_connection()

    def extract_axis_data(self, axis_data):
        """ Extrae información de las columnas (metadatos) de los datos del eje.
        
        Args:
            axis_data (list): Lista de diccionarios que representa los datos de los ejes de la respuesta de SAP.
        
        Returns:
            pandas.DataFrame: DataFrame con la información de las columnas como CHANM y sus etiquetas CAPTION.
        """
        column_info = []
        # Iterar a través de todos los datos de los ejes
        for data in axis_data:
            for set_item in data['SET']:
                # Agregar solo si el item es nuevo
                if not any(d['CHANM'] == set_item['CHANM'] for d in column_info):
                    column_info.append({
                        'CHANM': set_item['CHANM'],
                        'CAPTION': set_item['CAPTION']
                    })
    
        return pd.DataFrame(column_info)

    def extract_axis_info(self, axis_data):
        """
        Extrae y muestra información detallada sobre cada eje para ayudar a comprender la estructura del cubo.

        Args:
            axis_data (list): Lista de diccionarios que contienen información sobre los ejes.

        Returns:
            pd.DataFrame: Un DataFrame que contiene detalles sobre cada eje y sus características.
        """
        # Inicialización de la lista para almacenar la información de los ejes
        axis_info = []
        # Iteración a través de cada eje en los datos proporcionados
        for axis in axis_data:
            # Iteración a través de cada característica del eje
            for char in axis.get('CHARS', []):
                axis_info.append({
                    'AXIS': axis['AXIS'],
                    'CHANM': char['CHANM'],
                    'CAPTION': char['CAPTION'],
                    'CHATYP': char['CHATYP'],
                    'DETAILS': f"Presentaciones: {char['CHAPRSNT']}, Atributos: {len(char.get('ATTRINM', []))}"
                })
        # Creación de un DataFrame con la información recopilada
        return pd.DataFrame(axis_info)

    def clean_data(self, axis_data, cell_data):
        """
        Transforma los datos brutos de ejes y celdas del cubo SAP en un DataFrame estructurado.

        Args:
            axis_data (list): Lista que contiene detalles de los ejes (Columnas).
            cell_data (list): Lista que contiene los valores de las celdas relacionados con los ejes.

        Returns:
            pd.DataFrame: Un DataFrame estructurado que combina tanto los datos de los ejes como de las celdas.
        """
        # Extracción de detalles de los ejes
        data = []
        for entry in axis_data:
            if entry['AXIS'] == '001':  # ejemplo, ajustar basado en el caso de uso real
                for item in entry['SET']:
                    data.append({
                        'TUPLE_ORDINAL': item['TUPLE_ORDINAL'],
                        'CHANM': item['CHANM'],
                        'CAPTION': item['CAPTION'],
                        'CHAVL': item['CHAVL'],
                        'MONTH': item.get('CHAVL_EXT', '')  # campo de ejemplo
                    })
        
        # Creación de DataFrame a partir de los datos de los ejes
        df_final_data = pd.DataFrame(data)
        
        # Creación de DataFrame a partir de los datos de las celdas
        #df_final_cells = pd.DataFrame(cell_data)
        #df_final_cells = df_final_cells.rename(columns={'CELL_ORDINAL': 'TUPLE_ORDINAL'})
        
        # Fusión de los DataFrames en 'TUPLE_ORDINAL'
        #merged_df_final = pd.merge(df_final_data, df_final_cells, on='TUPLE_ORDINAL', how='left')
        
        return df_final_data
        #return merged_df_final #KEVIN

    def data_structuring(self, df_final, axis_info=None, values=['CAPTION']):
        """
        Organiza los datos en un formato estructurado, transformando filas repetidas en columnas.
        
        Args:
            df_final (pd.DataFrame): DataFrame que contiene los datos a organizar.
            axis_info (pd.DataFrame): DataFrame opcional que contiene información sobre los ejes para identificar las columnas dinámicamente.
            values (list): Lista con los nombres de las columnas que desea obtener del df_final
            
        Returns:
            pd.DataFrame: DataFrame organizado con filas repetidas transformadas en columnas.
        """
        if axis_info is not None:
            # Usamos la información del eje para identificar columnas CLAVE
            key_columns = axis_info[axis_info['CHATYP'] == '1']['CHANM'].tolist()
        
        # Filtrar el DataFrame original para mantener solo las columnas CLAVE y sus valores
        filtered_df_final = df_final[df_final['CHANM'].isin(key_columns)]
        
        # Usar pivot_table para manejar múltiples valores de 'values'
        pivot_df_final = filtered_df_final.pivot_table(index='TUPLE_ORDINAL', columns='CHANM', 
                                        values=values, 
                                        aggfunc='last').reset_index()

        # Aplanar las columnas Multindex resultantes
        pivot_df_final.columns = [' '.join(col).strip() for col in pivot_df_final.columns.values]

        # Construir diccionario de renombrado si axis_info está DISPONIBLE
        if axis_info is not None:
            rename_dict = {}
            for _, row in axis_info.iterrows():
                if row['CHANM'] in key_columns:
                    for suffix in values:
                        old_col_name = f"{suffix} {row['CHANM']}"
                        new_col_name = f"{row['CAPTION']}-{suffix}"
                        rename_dict[old_col_name] = new_col_name
            pivot_df_final = pivot_df_final.rename(columns=rename_dict)

        return pivot_df_final
    
    def extract_all_data(self, column_names: List[str],query_name, view_id, params) -> pd.DataFrame:

        self.open_connection()
        raw_data = self.execute_query(query_name, view_id, params)


        # Obtenemos diccionario con los nombres originales de las columnas
        axis_info = self.extract_axis_info(raw_data['E_AXIS_INFO'])
        # Obtenemos información combinada y transformada
        data_clean = self.clean_data(raw_data['E_AXIS_DATA'], raw_data['E_CELL_DATA'])
        # Estructuramos columnas y filas para un mejor entendimiento y visualización
        df_final_axis_values = self.data_structuring(data_clean, axis_info, ['CAPTION','CHAVL','VALUE']) #kevin
        
        for record in raw_data['E_CELL_DATA']:
            print(record)
        
        # Extraer los datos de las celdas del cubo y organizarlos en un DataFrame
        cell_records = [
            {'CELL_ORDINAL': record['CELL_ORDINAL'], 'VALUE': record['VALUE']}
            for record in raw_data['E_CELL_DATA']
        ]
        df_final_cell = pd.DataFrame(cell_records)

        df_final_cell['Group'] = df_final_cell.index // (len(column_names))

        # Generar el DataFrame con las columnas ordenadas de acuerdo a `column_names`
        df_final_cell_values = pd.DataFrame({
            name: df_final_cell.groupby('Group')['VALUE'].nth(i).values
            for i, name in enumerate(column_names)
        })
        return df_final_cell_values


## Consultar los datos de la vista en SAP

In [15]:
def get_data_sap():
    """
    Conecta a SAP y obtiene el stock del mes ANTERIOR al mes actual.
    """
    today = datetime.now()
    month = today.strftime("%m.%Y")

    sap_conn = SAPConnection(
        ashost=os.getenv("ASHOST"),
        sysnr=os.getenv("SYSNR"),
        client=os.getenv("CLIENT"),
        user=os.getenv("USER_SAP"),
        passwd=os.getenv("PASSWORD_SAP"),
        lang="ES",
    )

    params = [
        ("VAR_ID_6", "0I_CMNTH                      0004"),
        ("VAR_VALUE_LOW_EXT_6", month),
        ("VAR_VALUE_HIGH_EXT_6", month),
    ]

    result = sap_conn.execute_query("ZICM_CM03_Q001", "Z_BASE_RIESGO", params)

    # Primero extraemos la informacion de los ejes y los limpiamos
    axis_info = sap_conn.extract_axis_info(result["E_AXIS_INFO"])

    # Limpia los datos de la consulta y la informacion de las celdas
    data_clean = sap_conn.clean_data(result["E_AXIS_DATA"], result["E_CELL_DATA"])

    # Con la informacion de los ejes, estructuramos los datos para que se ajusten a la estructura de un DataFrame
    df_final_axis_values = sap_conn.data_structuring(
        data_clean, axis_info, ["CAPTION", "CHAVL"]
    )

    # Estos son los nombres de las columnas que se van a crear en el DataFrame final
    column_names = ['Costo Unitario Real',
                    'Inventario Disponibl', 
                    'Inventario No Dispon', 
                    'Valor OBSOLETO', 
                    'Valor BLOQUEADO MM', 
                    'Valor Total MM', 
                    'Permanencia'
                ]

    # Con la informacion de las celdas, creamos un diccionario que contiene los valores de cada celda
    cell_records = [
        {'CELL_ORDINAL': record['CELL_ORDINAL'], 'VALUE': record['VALUE']}
        for record in result['E_CELL_DATA']
    ]

    # Con el diccionario, creamos un DataFrame que contiene los valores de las celdas
    df_final_cell = pd.DataFrame(cell_records)

    # Agregamos una columna 'Group' que indica a que grupo pertenece cada celda. Se hace con // (divisin entera)
    df_final_cell['Group'] = df_final_cell.index // (len(column_names))

    # Creamos un nuevo DataFrame que contiene los valores de las celdas agrupados por el grupo
    df_final_cell_values = pd.DataFrame({
        name: df_final_cell.groupby('Group')['VALUE'].nth(i).values
        for i, name in enumerate(column_names)
    })

    # Finalmente, concatenamos los dos DataFrames en uno solo, con la informacion de los ejes y los valores de las celdas
    df_final_combined = pd.concat([df_final_axis_values, df_final_cell_values], axis=1)

    # Renombrar columnas
    df_final_combined = df_final_combined.rename(
        columns={
            'Material-CAPTION': 'Descripción', 
        }
    )

    # Eliminar columnas innecesarias
    df_final_combined = df_final_combined.drop(
        columns=[
            "TUPLE_ORDINAL",
            "Lote-CAPTION",
            "Fecha entrada-CAPTION",
            "Centro-CAPTION",
            "Unidad medida-CAPTION",
            "Codigo Almacen Cliente-CAPTION",
            "Rango Cobertura-CAPTION",
            "Creado el-CAPTION",
            "Fecha Bloqueado-CAPTION",
            "Fecha Obsoleto-CAPTION",
            "Fech. Fabricación-CAPTION",
            "Rango de Permanencia-CAPTION",
            "Rango Bloqueado-CAPTION",
            "Rango Vencidos-CAPTION",
            "Rango Obsoleto-CAPTION",
            "Rango Próximos a Ven-CAPTION",
            "Rango Próx.Vencer MM-CAPTION",
            "Fech, Caducidad/Fech Pref. Consumo-CAPTION",
            "Año natural/Mes-CHAVL",
            "Indicador Stock Espec.-CHAVL",
            "Marca de QM-CHAVL",
            "Tipo Material Inventario-CHAVL",
            "Negocio Inventarios-CHAVL",
            "Tipo de Material (I)-CHAVL",
            "Núm.stock.esp.-CHAVL",
        ]
    )

    # Eliminar sufijos de los nombres de columnas
    df_final_combined.columns = df_final_combined.columns.str.replace("-CHAVL", "").str.replace(
        "-CAPTION", ""
    )

    # Convertir todas las columnas a mayúsculas
    df_final_combined = df_final_combined.apply(
        lambda x: x.str.upper() if x.dtype == "object" else x
    )
    df_final_combined.columns = df_final_combined.columns.str.upper()

    # Convertir columnas numéricas
    columnas_numericas = [
        "COSTO UNITARIO REAL",
        "INVENTARIO DISPONIBL",
        "INVENTARIO NO DISPON",
        "VALOR OBSOLETO",
        "VALOR BLOQUEADO MM",
        "VALOR TOTAL MM",
        "PERMANENCIA",
    ]

    for col in columnas_numericas:
        if col in df_final_combined.columns:
            df_final_combined[col] = pd.to_numeric(df_final_combined[col], errors="coerce")

    # Convertir columnas de fechas
    columnas_fecha = [
        "FECHA ENTRADA",
        "CREADO EL",
        "FECHA BLOQUEADO",
        "FECHA OBSOLETO",
        "FECH. FABRICACIÓN",
        "FECH, CADUCIDAD/FECH PREF. CONSUMO",
    ]

    for col in columnas_fecha:
        if col in df_final_combined.columns:
            df_final_combined[col] = pd.to_datetime(
                df_final_combined[col], errors="coerce"
            ).dt.strftime("%d/%m/%Y")

    df_final_combined = pd.DataFrame(df_final_combined)

    return df_final_combined

def upload_dataframe_to_db(df_final_combined: pd.DataFrame) -> None:
    """
    Sube el DataFrame 'df_final_combined' a la base de datos en la tabla 'InventarioBaseRiesgo'.
    Se utiliza SQLAlchemy para establecer la conexión y el método to_sql de pandas para insertar
    todos los registros (append). Se asume que la tabla ya existe y que los nombres de columnas en 
    el DataFrame coinciden exactamente con los de la tabla en la base de datos.
    
    Args:
        df_final_combined (pd.DataFrame): DataFrame con las columnas y el orden requeridos.
    
    Returns:
        None
    """
    connection_string = (
            "mssql+pyodbc://{user}:{pwd}@{server}/{db}"
            "?driver=ODBC+Driver+17+for+SQL+Server"
        ).format(
            user=os.getenv("DB_USER"),
            pwd=os.getenv("DB_PASSWORD"),
            server=os.getenv("DB_SERVER"),
            db=os.getenv("DATABASE")
        )
    
    # Crear el engine de SQLAlchemy con fast_executemany habilitado
    engine = create_engine(connection_string, fast_executemany=True)
    
    try:
        # Insertar datos en la tabla InventarioBaseRiesgo. 
        # if_exists='append' se utiliza para agregar los datos sin reemplazar la tabla.
        df_final_combined.to_sql(
            name='InventarioBaseRiesgo',
            con=engine,
            if_exists='append',
            index=False,
            chunksize=1000  # Tamaño del chunk para inserciones masivas
        )
        print("Datos subidos correctamente a InventarioBaseRiesgo.")
    except Exception as e:
        print("Error al subir el DataFrame a la base de datos:", e)
    finally:
        engine.dispose()

def export_dataframe_to_excel(df: pd.DataFrame, filename: str = None) -> str:
    """
    Exporta un DataFrame a un archivo Excel con la fecha actual y lo guarda en una ruta específica.

    Args:
        df: DataFrame a exportar
        filename: Nombre base del archivo (opcional)

    Returns:
        str: Ruta del archivo Excel creado
    """
    # Definir la ruta de destino
    output_dir = r"C:\Users\prac.planeacionfi\OneDrive - Prebel S.A BIC\Escritorio\PRUEBAS BASE RIESGO"

    # Crear el nombre del archivo con la fecha actual
    current_date = datetime.now().strftime("%d-%m-%Y")
    if not filename:
        filename = f"Análisis_BaseRiesgo_Final_{current_date}.xlsx"

    # Crear la ruta completa
    file_path = os.path.join(output_dir, filename)

    # Asegurar que el directorio existe
    os.makedirs(output_dir, exist_ok=True)

    # Exportar a Excel
    try:
        df.to_excel(file_path, index=False, sheet_name="Base de Riesgo")
        print(f"Archivo Excel creado exitosamente: {file_path}")
        return file_path
    except Exception as e:
        print(f"Error al exportar a Excel: {str(e)}")
        return None

In [16]:
def filter_avon_natura(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filtra un DataFrame para quedarse solo con los materiales
    cuya 'MARCA DE QM' sea 'AVON' o 'NATURA'.

    Args:
        df (pd.DataFrame): DataFrame original que contiene la columna 'MARCA DE QM'.

    Returns:
        pd.DataFrame: Nuevo DataFrame con solo las filas de AVON y NATURA,
                      reindexado de 0 a N-1.
    """
    mask = df["MARCA DE QM"].isin(["AVON", "NATURA"])
    return df.loc[mask].reset_index(drop=True)

def filter_marca_otros(df: pd.DataFrame) -> pd.DataFrame:
    """
    Devuelve solo las filas donde 'MARCA DE QM' NO sea ni 'AVON' ni 'NATURA'.
    Esto permite trabajar con el resto de las marcas por separado.
    """
    # Aseguramos uniformidad en mayúsculas
    df = df.copy()
    df["MARCA DE QM"] = df["MARCA DE QM"].str.upper()
    
    # Filtramos inversamente
    mask = ~df["MARCA DE QM"].isin(["AVON", "NATURA"])
    return df[mask].reset_index(drop=True)

In [17]:
# 1. Obtienes el DataFrame completo
df_final_combined = get_data_sap()

# 2. Filtro para obtener solo los datos de AVON y NATURA
df_avon_natura = filter_avon_natura(df_final_combined)

# 3. Filtro para obtener los datos de otras marcas
df_otras_marcas = filter_marca_otros(df_final_combined)

🛠 Sending Parameters to SAP BW:
VAR_ID_6 = 0I_CMNTH                      0004
VAR_VALUE_LOW_EXT_6 = 05.2025
VAR_VALUE_HIGH_EXT_6 = 05.2025
✅ Query executed successfully!


C:\Users\prac.planeacionfi\AppData\Local\Temp\ipykernel_6420\600308896.py:143: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_final_combined[col] = pd.to_datetime(
C:\Users\prac.planeacionfi\AppData\Local\Temp\ipykernel_6420\600308896.py:143: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_final_combined[col] = pd.to_datetime(


In [18]:
def get_sql_engine():
    """
    Establece la conexión a SQL Server mediante SQLAlchemy.
    Configura los datos de conexión utilizando variables de entorno.
    """
    try:
        connection_string = (
            "mssql+pyodbc://{user}:{pwd}@{server}/{db}"
            "?driver=ODBC+Driver+17+for+SQL+Server"
        ).format(
            user=os.getenv("DB_USER"),
            pwd=os.getenv("DB_PASSWORD"),
            server=os.getenv("DB_SERVER"),
            db=os.getenv("DATABASE")
        )
        engine = create_engine(connection_string)
        print("Conexión exitosa a la base de datos.")
        return engine
    except Exception as e:
        print(f"Error al conectar a la base de datos: {str(e)}")
        raise

def execute_query(query):
    """
    Ejecuta el query en la base de datos y retorna un DataFrame utilizando SQLAlchemy.
    """
    engine = get_sql_engine()
    try:
        df = pd.read_sql(query, engine)
    except Exception as e:
        print(f"Error al ejecutar el query: {str(e)}")
        df = pd.DataFrame()  # Retorna un DataFrame vacío en caso de error
    finally:
        engine.dispose()  # Cierra la conexión
    return df

def get_inventario_matriz():
    """
    Extrae todos los registros de la tabla InventarioMatriz.
    """
    query = """
    SELECT 
        id_politica_base_riesgo,
        subsegmento,
        negocio,
        estado,
        cobertura
    FROM InventarioMatriz
    """
    return execute_query(query)

def get_matrices_base_riesgo():
    """
    Extrae todos los registros de la tabla MatrizBaseRiesgo.
    """
    query = """
    SELECT
        id_politica_base_riesgo,
        concatenado,
        segmento,
        permanencia,
        factor_prov,
        clasificacion,
        tipo_matriz
    FROM MatrizBaseRiesgo
    """
    return execute_query(query)

def df_matrices_merge():
    """
    Extrae los datos de ambas tablas, los unifica utilizando pd.merge() y convierte
    todos los campos de texto a mayúsculas de forma vectorizada.
    """
    df_inventario = get_inventario_matriz()
    df_matrices = get_matrices_base_riesgo()
    
    # Unificación utilizando la columna en común 'id_politica_base_riesgo'
    df_matrices_merge = pd.merge(df_inventario, df_matrices, 
                        on="id_politica_base_riesgo", 
                        how="inner")  # Cambia 'inner' por 'left' o 'outer' según lo requieras
    
    # Convertir los campos de tipo string a mayúsculas de forma vectorizada
    for col in df_matrices_merge.select_dtypes(include=["object"]).columns:
        df_matrices_merge[col] = df_matrices_merge[col].str.upper()
    
    # Convertir 'factor_prov' a float y normalizarlo
    df_matrices_merge['factor_prov'] = df_matrices_merge['factor_prov'].apply(lambda x: x/100)
    
    return df_matrices_merge

In [19]:
def df_matrices_avon_natura():
    """
    Extrae los datos de ambas tablas, las unifica utilizando pd.merge(),
    convierte todos los campos de texto a mayúsculas de forma vectorizada,
    normaliza el factor provisional y, finalmente, filtra solo las filas
    donde 'tipo_matriz' == 'MATRIZ NATURACO'.
    """
    # 1. Extraer los DataFrames base
    df_inventario = get_inventario_matriz()
    df_matrices   = get_matrices_base_riesgo()
    
    # 2. Unirlos por la clave foránea
    df = pd.merge(
        df_inventario,
        df_matrices,
        on="id_politica_base_riesgo",
        how="inner"  # o 'left' / 'outer' según necesidad
    )
    
    # 3. Pasar a mayúsculas todas las columnas de texto
    text_cols = df.select_dtypes(include="object").columns
    for c in text_cols:
        df[c] = df[c].str.upper()
    
    # 4. Normalizar el factor provisional (de porcentaje a [0–1])
    df["factor_prov"] = df["factor_prov"].astype(float) / 100.0
    
    # 5. Filtrar solo las filas de 'Matriz NaturaCo'
    #    (ten en cuenta que ya convertimos todo a mayúsculas)
    df = df[df["tipo_matriz"] == "MATRIZ NATURACO"].reset_index(drop=True)
    
    return df

def df_matrices_otros_tipos():
    """
    Devuelve solo las filas de df_merge donde 'tipo_matriz' sea distinto de 'Matriz NaturaCo'.
    Útil para aislar todas las demás matrices.
    """
    # 1. Extraer los DataFrames base
    df_inventario = get_inventario_matriz()
    df_matrices   = get_matrices_base_riesgo()
    
    # 2. Unirlos por la clave foránea
    df = pd.merge(
        df_inventario,
        df_matrices,
        on="id_politica_base_riesgo",
        how="inner"  # o 'left' / 'outer' según necesidad
    )
    
    # 3. Pasar a mayúsculas todas las columnas de texto
    text_cols = df.select_dtypes(include="object").columns
    for c in text_cols:
        df[c] = df[c].str.upper()
    
    # 4. Normalizar el factor provisional (de porcentaje a [0–1])
    df["factor_prov"] = df["factor_prov"].astype(float) / 100.0
    
    # Filtramos aquellas filas cuyo tipo de matriz NO sea 'MATRIZ NATURACO'
    mask = df["tipo_matriz"] != "MATRIZ NATURACO"
    
    return df[mask].reset_index(drop=True)

In [20]:
df_matrices_merge = df_matrices_merge()
df_matrices_avon_natura = df_matrices_avon_natura()
df_matrices_otros_tipos = df_matrices_otros_tipos()

Conexión exitosa a la base de datos.
Conexión exitosa a la base de datos.
Conexión exitosa a la base de datos.
Conexión exitosa a la base de datos.
Conexión exitosa a la base de datos.
Conexión exitosa a la base de datos.


# Lógica para AVON y NATURA

In [21]:
def insert_marks() -> Dict[str, str]:
    """
    Retorna un diccionario con el mapeo de marcas QM a marcas concatenadas.
    """
    # TODO: Implementar el diccionario de marcas según la lógica de negocio
    return {
        "ACCESORIOS": "ACCESORIOS",
        "ADIDAS": "ADIDAS",
        "AGATHA RUIZ DE LA PRADA": "AGATHA RUIZ DE LA PRADA",
        "ALICORP": "ALICORP",
        "AMAZON": "AMAZON",
        "AMWAY": "AMWAY",
        "ARDEN FOR MEN": "AFM/CFM",
        "AVON": "AVON",
        "BALANCE": "BALANCE",
        "BANCO PREBEL": "BANCO PREBEL",
        "BEAUTYHOLICS": "UTOPICK",
        "BIO OIL": "BIO OIL",
        "BIOTECNIK": "BIOTECNIK",
        "BURTS_BEES": "BURT'S BEES",
        "CADIVEU": "CADIVEU",
        "calculateA": "calculateA",
        "CATRICE": "CATRICE",
        "CONNECT FOR MEN": "AFM/CFM",
        "COSMETRIX": "COSMETRIX",
        "COVER GIRL": "COVER GIRL",
        "DIAL": "DIAL",
        "DOVE": "DOVE",
        "DYCLASS": "DYCLASS",
        "ECAR": "ECAR",
        "EL EXITO": "EL EXITO",
        "ELIZABETH ARDEN": "ELIZABETH ARDEN",
        "ESSENCE": "ESSENCE",
        "FAMILIA": "FAMILIA",
        "FEBREZE": "FEBREZE",
        "FISA": "FISA",
        "HASK": "HASK",
        "HENKEL": "HENKEL",
        "HERBAL ESSENCES": "HERBAL ESSENCES",
        "IMPORTADOS PROCTER": "IMPORTADOS PROCTER",
        "JERONIMO MARTINS": "JERONIMO MARTINS",
        "KANABECARE": "KANABECARE",
        "KIMBERLY": "KIMBERLY",
        "KOBA": "D1",
        "L&G ASOCIADOS": "L&G ASOCIADOS",
        "LA POPULAR": "LA POPULAR",
        "LEONISA": "LEONISA",
        "LOCATEL": "LOCATEL",
        "LOREAL": "LOREAL",
        "LOVE, BEAUTY AND PLANET": "LOVE, BEAUTY AND PLANET",
        "MAUI": "MAUI",
        "MAX FACTOR": "MAX FACTOR",
        "MAX FACTOR EXPORTACIÓN": "MAX FACTOR",
        "MAX FACTOR GLOBAL": "MAX FACTOR",
        "MF COL + EXP": "MAX FACTOR",
        "MF GLOBAL": "MAX FACTOR",
        "MILAGROS": "MILAGROS",
        "MONCLER": "MONCLER",
        "MORROCCANOIL": "MORROCCANOIL",
        "NATURA": "NATURA",
        "NATURAL PARADISE": "NATURAL PARADISE",
        "NIVEA": "NIVEA",
        "NOPIKEX": "NOPIKEX",
        "NOVAVENTA FPT": "NOVAVENTA FPT",
        "NUDE": "NUDE",
        "OGX": "OGX",
        "OLAY": "OLAY",
        "OMNILIFE": "OMNILIFE",
        "OTRAS": "OTRAS",
        "PREBEL": "PREBEL",
        "QVS": "ACCESORIOS",
        "SALLY HANSEN": "SALLY HANSEN",
        "SIN ASIGNAR": "SIN ASIGNAR",
        "SOLLA": "SOLLA",
        "ST. IVES": "ST. IVES",
        "UBU": "ACCESORIOS",
        "UNILEVER": "UNILEVER",
        "VENTA DIRECTA COSMÉTICOS": "VENTA DIRECTA COSMÉTICOS",
        "VITÚ": "VITÚ",
        "VITÚ  EXPORTACIÓN": "VITÚ",
        "WELLA CONSUMO": "WELLA CONSUMO",
        "WELLA PROFESSIONAL": "WELLA PROFESSIONAL",
        "YARDLEY": "YARDLEY",
        "CATÁLOGO DE PRODUCTOS": "CATÁLOGO DE PRODUCTOS",
        "D1": "D1",
        "WORMSER": "WORMSER",
        "PROCTER AND GAMBLE": "P&G",
        "DAVINES": "DAVINES",
        "LA FABRIL": "LA FABRIL",
        "REVOX": "REVOX",
        "TENDENCIAS AB": "TENDENCIAS AB",
    }


def insert_subsegmentacion() -> Dict[str, str]:


    """
    RETORNA UN DICCIONARIO CON EL MAPEO DE MARCAS QM A SUBSEGMENTACIÓN.
    """
    return {
        "ACCESORIOS": "OTROS",
        "ADIDAS": "OTROS",
        "AGATHA RUIZ DE LA PRADA": "OTROS",
        "ALICORP": "FULL",
        "AMAZON": "RETAILERS",
        "AMWAY": "SISTEMA DE VENTAS",
        "ARDEN FOR MEN": "OTROS",
        "AVON": "FULL",
        "BALANCE": "FULL",
        "BANCO PREBEL": "FULL",
        "BEAUTYHOLICS": "OTROS",
        "BIO OIL": "OTROS",
        "BIOTECNIK": "FULL",
        "BURTS_BEES": "OTROS",
        "CADIVEU": "PROFESIONALES",
        "CALA": "FULL",
        "CATRICE": "OTROS",
        "CONNECT FOR MEN": "OTROS",
        "COSMETRIX": "OTROS",
        "COVER GIRL": "OTROS",
        "DIAL": "RETAILERS",
        "DOVE": "OTROS",
        "DYCLASS": "SISTEMA DE VENTAS",
        "ECAR": "FULL",
        "EL EXITO": "RETAILERS",
        "ELIZABETH ARDEN": "OTROS",
        "ESSENCE": "OTROS",
        "FAMILIA": "FULL",
        "FEBREZE": "OTROS",
        "FISA": "FULL",
        "HASK": "OTROS",
        "HENKEL": "FULL",
        "HERBAL ESSENCES": "OTROS",
        "IMPORTADOS PROCTER": "OTROS",
        "JERONIMO MARTINS": "RETAILERS",
        "KANABECARE": "OTROS",
        "KIMBERLY": "FULL",
        "KOBA": "RETAILERS",
        "L&G ASOCIADOS": "FULL",
        "LA POPULAR": "RETAILERS",
        "LEONISA": "SISTEMA DE VENTAS",
        "LOCATEL": "RETAILERS",
        "LOREAL": "FULL",
        "LOVE, BEAUTY AND PLANET": "OTROS",
        "MAUI": "OTROS",
        "MAX FACTOR": "OTROS",
        "MAX FACTOR EXPORTACIÓN": "OTROS",
        "MAX FACTOR GLOBAL": "OTROS",
        "MILAGROS": "SISTEMA DE VENTAS",
        "MONCLER": "RETAILERS",
        "MORROCCANOIL": "PROFESIONALES",
        "NATURA": "FULL",
        "NATURAL PARADISE": "OTROS",
        "NIVEA": "FULL",
        "NOPIKEX": "OTROS",
        "NOVAVENTA FPT": "SISTEMA DE VENTAS",
        "NUDE": "NUDE",
        "OGX": "OTROS",
        "OLAY": "OTROS",
        "OMNILIFE": "SISTEMA DE VENTAS",
        "OTRAS": "OTROS",
        "PREBEL": "OTROS",
        "QVS": "OTROS",
        "SALLY HANSEN": "OTROS",
        "SIN ASIGNAR": "FULL",
        "SOLLA": "FULL",
        "ST. IVES": "OTROS",
        "UBU": "OTROS",
        "UNILEVER": "TOLL",
        "VENTA DIRECTA COSMÉTICOS": "FULL",
        "VITÚ": "OTROS",
        "VITÚ  EXPORTACIÓN": "OTROS",
        "WELLA CONSUMO": "OTROS",
        "WELLA PROFESSIONAL": "PROFESIONALES",
        "YARDLEY": "OTROS",
        "D1": "RETAILERS",
        "CATÁLOGO DE PRODUCTOS": "RETAILERS",
        "WORMSER": "RETAILERS",
        "PROCTER AND GAMBLE": "FULL",
        "DAVINES": "OTROS",
        "LA FABRIL": "OTROS",
        "REVOX": "OTROS",
        "TENDENCIAS AB": "RETAILERS"
    }


def insert_segments() -> Dict[str, str]:
    """
    Retorna un diccionario con el mapeo de marcas QM a segmentaciones.
    """
    # TODO: Implementar el diccionario de segmentaciones según la lógica de negocio
    return {
        "OTROS CLIENTES DO": "DUEÑOS DE CANAL",
        "MARKETING PERSONAL": "DUEÑOS DE CANAL",
        "OMNILIFE": "DUEÑOS DE CANAL",
        "JERONIMO MARTINS": "DUEÑOS DE CANAL",
        "LEONISA": "DUEÑOS DE CANAL",
        "LOCATEL": "DUEÑOS DE CANAL",
        "NOVAVENTA": "DUEÑOS DE CANAL",
        "EL ÉXITO": "DUEÑOS DE CANAL",
        "MILAGROS ENTERPRISE": "DUEÑOS DE CANAL",
        "LA POPULAR": "DUEÑOS DE CANAL",
        "D1": "DUEÑOS DE CANAL",
        "USA": "DUEÑOS DE CANAL",
        "USA": "DUEÑOS DE CANAL",
        "EL EXITO": "DUEÑOS DE CANAL",
        "NOVAVENTA FPT": "DUEÑOS DE CANAL",
        "MILAGROS": "DUEÑOS DE CANAL",
        "WORMSER": "DUEÑOS DE CANAL",
        "TENDENCIAS AB": "DUEÑOS DE CANAL",
        "LA FABRIL": "DUEÑOS DE CANAL",
        "UNILEVER": "EXPERTOS LOCALES",
        "NATURA": "EXPERTOS LOCALES",
        "BIOTECNIK": "EXPERTOS LOCALES",
        "NIVEA": "EXPERTOS LOCALES",
        "BRITO": "EXPERTOS LOCALES",
        "AVON": "EXPERTOS LOCALES",
        "OTROS EXPERTOS LOCALES": "EXPERTOS LOCALES",
        "ALICORP": "EXPERTOS LOCALES",
        "SOLLA": "EXPERTOS LOCALES",
        "ECAR": "EXPERTOS LOCALES",
        "FISA": "EXPERTOS LOCALES",
        "KIMBERLY": "EXPERTOS LOCALES",
        "BELCORP": "EXPERTOS LOCALES",
        "AMWAY": "EXPERTOS LOCALES",
        "PROCTER AND GAMBLE": "EXPERTOS LOCALES",
        "HENKEL": "EXPERTOS LOCALES",
        "DIAL": "EXPERTOS LOCALES",
        "BEIERSDORF": "EXPERTOS LOCALES",
        "OTROS EXPERTOS LOCALES": "EXPERTOS LOCALES",
        "FAMILIA": "EXPERTOS LOCALES",
        "BALANCE": "EXPERTOS LOCALES",
        "MAX FACTOR": "EXPERTOS NO LOCALES",
        "DYCLASS": "EXPERTOS NO LOCALES",
        "WELLA CONSUMO": "EXPERTOS NO LOCALES",
        "BIO OIL": "EXPERTOS NO LOCALES",
        "OGX": "EXPERTOS NO LOCALES",
        "COVER GIRL": "EXPERTOS NO LOCALES",
        "WELLA PROFESSIONAL": "EXPERTOS NO LOCALES",
        "ADIDAS": "EXPERTOS NO LOCALES",
        "ACCESORIOS": "EXPERTOS NO LOCALES",
        "BURTS_BEES": "EXPERTOS NO LOCALES",
        "NOPIKEX": "EXPERTOS NO LOCALES",
        "QVS": "EXPERTOS NO LOCALES",
        "UBU": "EXPERTOS NO LOCALES",
        "ESSENCE": "EXPERTOS NO LOCALES",
        "MORROCCANOIL": "EXPERTOS NO LOCALES",
        "HASK": "EXPERTOS NO LOCALES",
        "HERBAL ESSENCES": "EXPERTOS NO LOCALES",
        "LOVE, BEAUTY AND PLANET": "EXPERTOS NO LOCALES",
        "CATRICE": "EXPERTOS NO LOCALES",
        "NATURAL PARADISE": "EXPERTOS NO LOCALES",
        "OLAY": "EXPERTOS NO LOCALES",
        "MID": "EXPERTOS NO LOCALES",
        "SECRET": "EXPERTOS NO LOCALES",
        "FEBREZE": "EXPERTOS NO LOCALES",
        "TAMPAX": "EXPERTOS NO LOCALES",
        "OFCORSS C.I HERMECO": "EXPERTOS NO LOCALES",
        "CADIVEU": "EXPERTOS NO LOCALES",
        "MAX FACTOR GLOBAL": "EXPERTOS NO LOCALES",
        "SEBASTIAN": "EXPERTOS NO LOCALES",
        "AFFRESH": "EXPERTOS NO LOCALES",
        "COSMETRIX": "EXPERTOS NO LOCALES",
        "INCENTIVOS MAX FACTOR": "EXPERTOS NO LOCALES",
        "OTROS EXPERTOS NO LOCALES": "EXPERTOS NO LOCALES",
        "DAVINES": "EXPERTOS NO LOCALES",
        "P&G": "EXPERTOS NO LOCALES",
        "REVOX": "EXPERTOS NO LOCALES",
        "UTOPICK": "EXPERTOS NO LOCALES",
        "IMPORTADOS PROCTER": "EXPERTOS NO LOCALES",
        "ST. IVES": "EXPERTOS NO LOCALES",
        "ARDEN FOR MEN": "MARCAS PROPIAS",
        "NUDE": "MARCAS PROPIAS",
        "ELIZABETH ARDEN": "MARCAS PROPIAS",
        "YARDLEY": "MARCAS PROPIAS",
        "VITÚ": "MARCAS PROPIAS",
        "PREBEL": "MARCAS PROPIAS",
        "OTRAS MP": "MARCAS PROPIAS",
        "AFM/CFM": "MARCAS PROPIAS",
        "BODY CLEAR": "NO APLICA",
        "OTRAS": "NO APLICA",
        "GILLETTE": "NO APLICA",
        "L&G ASOCIADOS": "NO APLICA",
        "CATÁLOGO DE PRODUCTOS": "NO APLICA",
        "HINODE": "NO APLICA",
        "PFIZER": "NO APLICA",
        "CONTEXPORT DISNEY": "NO APLICA",
        "SYSTEM PROFESSIONAL": "NO APLICA",
        "WELONDA": "NO APLICA",
        "SIN ASIGNAR": "NO APLICA",
        "CATÁLOGO DE PRODUCTOS": "NO APLICA",
    }


def calculate_rango_permanencia(row: pd.Series) -> str:
    """
    Calcula el rango de permanencia basado en las condiciones especificadas.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de permanencia calculado
    """
    lote = row.get("LOTE", None)
    permanencia = row.get("PERMANENCIA", None)
    rango_permanencia = row.get("RANGO DE PERMANENCIA", None)

    if lote == "222222":
        return "1.MENOR DE 90 DIAS"
    elif permanencia == 0 and rango_permanencia == "5.MAYOR O IGUAL A 360 DIAS":
        return "5.ENTRE 360 Y 540 DIAS"
    elif rango_permanencia == "5.MAYOR O IGUAL A 360 DIAS":
        if permanencia < 540:
            return "5.ENTRE 360 Y 540 DIAS"
        elif permanencia < 720:
            return "6.ENTRE 540 Y 720 DIAS"
        else:
            return "7.MAYOR DE 720 DIAS"
    else:
        return rango_permanencia


def calculate_status_cons(row: pd.Series) -> str:
    """
    Calcula el estado de consumo basado en las condiciones de vencimiento, bloqueo y obsolescencia.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Estado calculado (VENCIDO, BLOQUEADO, OBSOLETO, PAV o DISPONIBLE)
    """
    rango_prox_vencer = row.get("RANGO PRÓX.VENCER MM")
    valor_BLOQUEADO = row.get("VALOR BLOQUEADO MM")
    valor_OBSOLETO = row.get("VALOR OBSOLETO")

    if rango_prox_vencer == "VENCIDO":
        return "VENCIDO"
    elif valor_BLOQUEADO != 0:
        return "BLOQUEADO"
    elif valor_OBSOLETO != 0:
        return "OBSOLETO"
    elif rango_prox_vencer in ["1.PAV 3 MESES", "2.PAV 4 A 6 MESES"]:
        return "PAV"
    else:
        return "DISPONIBLE"


def calculate_valor_def(row: pd.Series) -> float:
    """
    Calcula el valor definitivo basado en el estado de consumo.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        float: Valor definitivo calculado
    """
    status_cons = row.get("STATUS CONS")
    valor_BLOQUEADO = row.get("VALOR BLOQUEADO MM")
    valor_total = row.get("VALOR TOTAL MM")

    return valor_BLOQUEADO if status_cons == "BLOQUEADO" else valor_total


def calculate_rango_obsolescencia(row: pd.Series) -> str:
    """
    Calcula el rango de obsolescencia basado en las fechas de entrada y obsolescencia.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de obsolescencia calculado
    """
    status_cons = row.get("STATUS CONS")
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_OBSOLETO = row.get("FECHA OBSOLETO")

    if status_cons != "OBSOLETO" or pd.isna(fecha_entrada) or pd.isna(fecha_OBSOLETO):
        return "FALSO"

    dias_obsolescencia = (fecha_entrada - fecha_OBSOLETO).days

    if dias_obsolescencia <= 90:
        return "1.MENOR DE 90 DIAS"
    elif 90 < dias_obsolescencia <= 180:
        return "2.ENTRE 90 Y 180 DIAS"
    elif 180 < dias_obsolescencia <= 270:
        return "3.ENTRE 180 Y 270 DIAS"
    elif 270 < dias_obsolescencia <= 360:
        return "4.ENTRE 270 Y 360 DIAS"
    elif 360 < dias_obsolescencia <= 540:
        return "5.ENTRE 360 Y 540 DIAS"
    elif 540 < dias_obsolescencia <= 720:
        return "6.ENTRE 540 Y 720 DIAS"
    else:
        return "7.MAYOR DE 720 DIAS"


def calculate_rango_vencido(row: pd.Series) -> str:
    """
    Calcula el rango de vencimiento basado en las fechas de entrada y caducidad.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de vencimiento calculado
    """
    status_cons = row.get("STATUS CONS")
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_caducidad = row.get("FECH, CADUCIDAD/FECH PREF. CONSUMO")

    if status_cons != "VENCIDO" or pd.isna(fecha_entrada) or pd.isna(fecha_caducidad):
        return "FALSO"

    dias_vencido = (fecha_entrada - fecha_caducidad).days

    if dias_vencido <= 90:
        return "1.MENOR DE 90 DIAS"
    elif 90 < dias_vencido <= 180:
        return "2.ENTRE 90 Y 180 DIAS"
    elif 180 < dias_vencido <= 270:
        return "3.ENTRE 180 Y 270 DIAS"
    elif 270 < dias_vencido <= 360:
        return "4.ENTRE 270 Y 360 DIAS"
    elif 360 < dias_vencido <= 540:
        return "5.ENTRE 360 Y 540 DIAS"
    elif 540 < dias_vencido <= 720:
        return "6.ENTRE 540 Y 720 DIAS"
    else:
        return "7.MAYOR DE 720 DIAS"


def calculate_rango_bloqueado(row: pd.Series) -> str:
    """
    Calcula el rango de bloqueo basado en las fechas de entrada y bloqueo.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de bloqueo calculado
    """
    status_cons = row.get("STATUS CONS")
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_bloqueado = row.get("FECHA BLOQUEADO")

    if status_cons != "BLOQUEADO" or pd.isna(fecha_entrada) or pd.isna(fecha_bloqueado):
        return "FALSO"

    dias_vencido = (fecha_entrada - fecha_bloqueado).days

    if dias_vencido <= 90:
        return "1.MENOR DE 90 DIAS"
    elif 90 < dias_vencido <= 180:
        return "2.ENTRE 90 Y 180 DIAS"
    elif 180 < dias_vencido <= 270:
        return "3.ENTRE 180 Y 270 DIAS"
    elif 270 < dias_vencido <= 360:
        return "4.ENTRE 270 Y 360 DIAS"
    elif 360 < dias_vencido <= 540:
        return "5.ENTRE 360 Y 540 DIAS"
    elif 540 < dias_vencido <= 720:
        return "6.ENTRE 540 Y 720 DIAS"
    else:
        return "7.MAYOR A 720 DIAS"


def calculate_tiempo_bloqueo(row: pd.Series) -> str:
    """
    Calcula el tiempo de bloqueo basado en las fechas de entrada y bloqueo.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Tiempo de bloqueo calculado
    """
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_bloqueado = row.get("FECHA BLOQUEADO")
    
    if pd.isna(fecha_entrada) or pd.isna(fecha_bloqueado):
        return 0

    tiempo_bloqueado = (fecha_entrada - fecha_bloqueado).days
    
    return tiempo_bloqueado    


def calculate_rango_cons(row: pd.Series) -> str:
    """
    Calcula el rango de consumo final basado en el status y los rangos correspondientes.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de consumo calculado
    """
    status = row.get("STATUS CONS")

    if status == "OBSOLETO":
        return row.get("RANGO OBSOLESCENCIA")
    elif status == "VENCIDO":
        return row.get("RANGO VENCIDO 2")
    elif status == "BLOQUEADO":
        return row.get("RANGO BLOQUEADO 2")
    elif status == "PAV":
        return row.get("RANGO DE PERMANENCIA 2")
    else:
        return row.get("RANGO DE PERMANENCIA 2")


def calculate_avon_natura_factor_and_class(row, lookup_dict):
    """
    Calcula factor provisional y clasificación solo para materiales de AVON y NATURA,
    siguiendo la lógica de la fórmula de Excel proporcionada.
    """
    seg        = row["SEGMENTACION"]
    status     = row["STATUS CONS"]
    tiempo     = row["TIEMPO BLOQUEO"]
    perm       = row["PERMANENCIA"]
    tipo_mat   = row["TIPO DE MATERIAL (I)"]
    negocio    = row["NEGOCIO INVENTARIOS"]
    rango_cons = str(row["RANGO CONS"]).strip()
    indic      = row["INDICADOR STOCK ESPEC."]

    # 1) Segmento interno bloqueado poco tiempo
    if seg in ["MARCAS PROPIAS", "EXPERTOS NO LOCALES", "DUEÑOS DE DEMANDA"] \
       and status == "BLOQUEADO" and tiempo <= 30:
        return 0.0, "BAJO"

    # 2) Disponible o PAV y permanencia <= 30 en granel
    if status in ["DISPONIBLE", "PAV"] \
       and perm <= 30 \
       and tipo_mat in ["GRANEL FAB A TERCERO", "GRANEL"]:
        return 0.0, "BAJO"

    # 3) Negocio FPT → lookup con A2&AQ2&AV2
    if negocio == "FPT":
        key = f"{negocio}{status}{rango_cons}"
        return lookup_dict.get(key, (0.0, "BAJO"))

    # 4) Stock ≠ W y obsoleto/bloqueado/vencido → mismo lookup
    if indic != "W" and status in ["OBSOLETO", "BLOQUEADO", "VENCIDO"]:
        key = f"{negocio}{status}{rango_cons}"
        return lookup_dict.get(key, (0.0, "BAJO"))

    # 5) Stock ≠ W y disponible → lookup con cobertura+permanencia2
    if indic != "W" and status == "DISPONIBLE":
        key = (
            str(row["RANGO COBERTURA"]).strip()
            + str(row["RANGO DE PERMANENCIA 2"]).strip()
        )
        return lookup_dict.get(key, (0.0, "BAJO"))

    # 6) Marca Avon/Natura y estado vencido/obsoleto/PAV → primer dígito
    marca_qm = row["MARCA DE QM"]
    if marca_qm in ["AVON", "NATURA"] and status in ["VENCIDO", "OBSOLETO", "PAV"]:
        try:
            first_digit = int(rango_cons[0])
            return (1.0, "MUY ALTO") if first_digit > 4 else (0.2, "MEDIO")
        except:
            return 0.0, "BAJO"

    # Default
    return 0.0, "BAJO"


def calculate_base_riesgo_column(row: pd.Series) -> float:
    """
    Calcula el valor de la columna de base riesgo
    """
    if row["CLAS BASE RIESGO"] == "BAJO":
        return 0.0
    else:
        return row["VALOR DEF"]


def calculate_provision_column(row: pd.Series) -> float:
    """
    Calcula el valor de la columna de provisión
    """
    if row["MARCA DE QM"] == "OTRAS":
        return 0.0
    else:
        return row["VALOR DEF"] * row['FACTOR PROV']


def process_dataframe_avon_natura(df_avon_natura: pd.DataFrame,df_matrices_avon_natura: pd.DataFrame) -> pd.DataFrame:
    """
    Procesa el DataFrame aplicando todas las reglas de negocio en el orden específico requerido.

    Args:
        df: DataFrame con los datos de SAP

    Returns:
        pd.DataFrame: DataFrame procesado con todas las columnas calculadas
    """
    # 1. Añadir columnas formuladas de 'MARCA CONCAT' y 'SEGMENTACION'
    df_avon_natura["MARCA CONCAT"] = df_avon_natura["MARCA DE QM"].apply(lambda x: insert_marks().get(x, ""))
    df_avon_natura["SEGMENTACION"] = df_avon_natura["MARCA DE QM"].apply(
        lambda x: insert_segments().get(x, "OTRAS")
    )
    df_avon_natura["SUBSEGMENTACION"] = df_avon_natura["MARCA DE QM"].apply(
        lambda x: insert_subsegmentacion().get(x, "")
    )

    # 2. Calcular 'RANGO DE PERMANENCIA 2'
    required_columns = {"LOTE", "PERMANENCIA", "RANGO DE PERMANENCIA"}
    if required_columns.issubset(df_avon_natura.columns):
        df_avon_natura["RANGO DE PERMANENCIA 2"] = df_avon_natura.apply(calculate_rango_permanencia, axis=1)
    else:
        print(f"Faltan columnas: {required_columns - set(df_avon_natura.columns)}")

    # 3. Calcular 'STATUS CONS'
    required_columns = {"RANGO PRÓX.VENCER MM", "VALOR BLOQUEADO MM", "VALOR OBSOLETO"}
    if required_columns.issubset(df_avon_natura.columns):
        df_avon_natura["STATUS CONS"] = df_avon_natura.apply(calculate_status_cons, axis=1)
    else:
        print(f"Faltan columnas: {required_columns - set(df_avon_natura.columns)}")

    # 4. Calcular 'VALOR DEF'
    required_columns = {"STATUS CONS", "VALOR BLOQUEADO MM", "VALOR TOTAL MM"}
    if required_columns.issubset(df_avon_natura.columns):
        df_avon_natura["VALOR DEF"] = df_avon_natura.apply(calculate_valor_def, axis=1)
    else:
        print(f"Faltan columnas: {required_columns - set(df_avon_natura.columns)}")

    # 5. Reemplazar valores inválidos
    df_avon_natura.replace("#", np.nan, inplace=True)

    # 6. Convertir columnas de fecha
    date_columns = [
        "FECHA ENTRADA",
        "FECHA OBSOLETO",
        "FECHA BLOQUEADO",
        "FECH. FABRICACIÓN",
        "CREADO EL",
        "FECH, CADUCIDAD/FECH PREF. CONSUMO",
    ]

    for col in date_columns:
        if col in df_avon_natura.columns:
            df_avon_natura[col] = pd.to_datetime(df_avon_natura[col], format="%d/%m/%Y", errors="coerce")

    # 7. Calcular 'RANGO OBSOLESCENCIA'
    df_avon_natura["RANGO OBSOLESCENCIA"] = df_avon_natura.apply(calculate_rango_obsolescencia, axis=1)

    # 8. Calcular 'RANGO VENCIDO 2'
    df_avon_natura["RANGO VENCIDO 2"] = df_avon_natura.apply(calculate_rango_vencido, axis=1)

    # 9. Calcular 'RANGO BLOQUEADO 2'
    df_avon_natura["RANGO BLOQUEADO 2"] = df_avon_natura.apply(calculate_rango_bloqueado, axis=1)

    # 10. Calcular 'RANGO CONS'
    df_avon_natura["RANGO CONS"] = df_avon_natura.apply(calculate_rango_cons, axis=1)
    
    # 11. Calcular 'TIEMPO BLOQUEO'
    df_avon_natura["TIEMPO BLOQUEO"] = df_avon_natura.apply(calculate_tiempo_bloqueo, axis=1)

    # Construir lookup_dict **una vez** antes del apply
    lookup_dict = {
        str(r["concatenado"]).strip(): (r["factor_prov"], r["clasificacion"])
        for _, r in df_matrices_avon_natura.iterrows()
    }

    # Aplicar fila a fila y asignar dos nuevas columnas
    df_avon_natura[["FACTOR PROV", "CLAS BASE RIESGO"]] = df_avon_natura.apply(
        lambda row: pd.Series(calculate_avon_natura_factor_and_class(row, lookup_dict)),
        axis=1
    )

    # 13. BASE RIESGO
    df_avon_natura["BASE RIESGO"] = df_avon_natura.apply(
        calculate_base_riesgo_column, axis=1
    )
    # 14. PROVISION
    df_avon_natura["PROVISION"] = df_avon_natura.apply(
        calculate_provision_column, axis=1
    )

    return df_avon_natura


In [22]:
df_final_avon_natura = process_dataframe_avon_natura(df_avon_natura, df_matrices_avon_natura)

# Lógica para el resto de marcas

In [23]:
def insert_marks() -> Dict[str, str]:
    """
    Retorna un diccionario con el mapeo de marcas QM a marcas concatenadas.
    """
    # TODO: Implementar el diccionario de marcas según la lógica de negocio
    return {
        "ACCESORIOS": "ACCESORIOS",
        "ADIDAS": "ADIDAS",
        "AGATHA RUIZ DE LA PRADA": "AGATHA RUIZ DE LA PRADA",
        "ALICORP": "ALICORP",
        "AMAZON": "AMAZON",
        "AMWAY": "AMWAY",
        "ARDEN FOR MEN": "AFM/CFM",
        "AVON": "AVON",
        "BALANCE": "BALANCE",
        "BANCO PREBEL": "BANCO PREBEL",
        "BEAUTYHOLICS": "UTOPICK",
        "BIO OIL": "BIO OIL",
        "BIOTECNIK": "BIOTECNIK",
        "BURTS_BEES": "BURT'S BEES",
        "CADIVEU": "CADIVEU",
        "calculateA": "calculateA",
        "CATRICE": "CATRICE",
        "CONNECT FOR MEN": "AFM/CFM",
        "COSMETRIX": "COSMETRIX",
        "COVER GIRL": "COVER GIRL",
        "DIAL": "DIAL",
        "DOVE": "DOVE",
        "DYCLASS": "DYCLASS",
        "ECAR": "ECAR",
        "EL EXITO": "EL EXITO",
        "ELIZABETH ARDEN": "ELIZABETH ARDEN",
        "ESSENCE": "ESSENCE",
        "FAMILIA": "FAMILIA",
        "FEBREZE": "FEBREZE",
        "FISA": "FISA",
        "HASK": "HASK",
        "HENKEL": "HENKEL",
        "HERBAL ESSENCES": "HERBAL ESSENCES",
        "IMPORTADOS PROCTER": "IMPORTADOS PROCTER",
        "JERONIMO MARTINS": "JERONIMO MARTINS",
        "KANABECARE": "KANABECARE",
        "KIMBERLY": "KIMBERLY",
        "KOBA": "D1",
        "L&G ASOCIADOS": "L&G ASOCIADOS",
        "LA POPULAR": "LA POPULAR",
        "LEONISA": "LEONISA",
        "LOCATEL": "LOCATEL",
        "LOREAL": "LOREAL",
        "LOVE, BEAUTY AND PLANET": "LOVE, BEAUTY AND PLANET",
        "MAUI": "MAUI",
        "MAX FACTOR": "MAX FACTOR",
        "MAX FACTOR EXPORTACIÓN": "MAX FACTOR",
        "MAX FACTOR GLOBAL": "MAX FACTOR",
        "MF COL + EXP": "MAX FACTOR",
        "MF GLOBAL": "MAX FACTOR",
        "MILAGROS": "MILAGROS",
        "MONCLER": "MONCLER",
        "MORROCCANOIL": "MORROCCANOIL",
        "NATURA": "NATURA",
        "NATURAL PARADISE": "NATURAL PARADISE",
        "NIVEA": "NIVEA",
        "NOPIKEX": "NOPIKEX",
        "NOVAVENTA FPT": "NOVAVENTA FPT",
        "NUDE": "NUDE",
        "OGX": "OGX",
        "OLAY": "OLAY",
        "OMNILIFE": "OMNILIFE",
        "OTRAS": "OTRAS",
        "PREBEL": "PREBEL",
        "QVS": "ACCESORIOS",
        "SALLY HANSEN": "SALLY HANSEN",
        "SIN ASIGNAR": "SIN ASIGNAR",
        "SOLLA": "SOLLA",
        "ST. IVES": "ST. IVES",
        "UBU": "ACCESORIOS",
        "UNILEVER": "UNILEVER",
        "VENTA DIRECTA COSMÉTICOS": "VENTA DIRECTA COSMÉTICOS",
        "VITÚ": "VITÚ",
        "VITÚ  EXPORTACIÓN": "VITÚ",
        "WELLA CONSUMO": "WELLA CONSUMO",
        "WELLA PROFESSIONAL": "WELLA PROFESSIONAL",
        "YARDLEY": "YARDLEY",
        "CATÁLOGO DE PRODUCTOS": "CATÁLOGO DE PRODUCTOS",
        "D1": "D1",
        "WORMSER": "WORMSER",
        "PROCTER AND GAMBLE": "P&G",
        "DAVINES": "DAVINES",
        "LA FABRIL": "LA FABRIL",
        "REVOX": "REVOX",
        "TENDENCIAS AB": "TENDENCIAS AB",
    }


def insert_subsegmentacion() -> Dict[str, str]:


    """
    RETORNA UN DICCIONARIO CON EL MAPEO DE MARCAS QM A SUBSEGMENTACIÓN.
    """
    return {
        "ACCESORIOS": "OTROS",
        "ADIDAS": "OTROS",
        "AGATHA RUIZ DE LA PRADA": "OTROS",
        "ALICORP": "FULL",
        "AMAZON": "RETAILERS",
        "AMWAY": "SISTEMA DE VENTAS",
        "ARDEN FOR MEN": "OTROS",
        "AVON": "FULL",
        "BALANCE": "FULL",
        "BANCO PREBEL": "FULL",
        "BEAUTYHOLICS": "OTROS",
        "BIO OIL": "OTROS",
        "BIOTECNIK": "FULL",
        "BURTS_BEES": "OTROS",
        "CADIVEU": "PROFESIONALES",
        "CALA": "FULL",
        "CATRICE": "OTROS",
        "CONNECT FOR MEN": "OTROS",
        "COSMETRIX": "OTROS",
        "COVER GIRL": "OTROS",
        "DIAL": "RETAILERS",
        "DOVE": "OTROS",
        "DYCLASS": "SISTEMA DE VENTAS",
        "ECAR": "FULL",
        "EL EXITO": "RETAILERS",
        "ELIZABETH ARDEN": "OTROS",
        "ESSENCE": "OTROS",
        "FAMILIA": "FULL",
        "FEBREZE": "OTROS",
        "FISA": "FULL",
        "HASK": "OTROS",
        "HENKEL": "FULL",
        "HERBAL ESSENCES": "OTROS",
        "IMPORTADOS PROCTER": "OTROS",
        "JERONIMO MARTINS": "RETAILERS",
        "KANABECARE": "OTROS",
        "KIMBERLY": "FULL",
        "KOBA": "RETAILERS",
        "L&G ASOCIADOS": "FULL",
        "LA POPULAR": "RETAILERS",
        "LEONISA": "SISTEMA DE VENTAS",
        "LOCATEL": "RETAILERS",
        "LOREAL": "FULL",
        "LOVE, BEAUTY AND PLANET": "OTROS",
        "MAUI": "OTROS",
        "MAX FACTOR": "OTROS",
        "MAX FACTOR EXPORTACIÓN": "OTROS",
        "MAX FACTOR GLOBAL": "OTROS",
        "MILAGROS": "SISTEMA DE VENTAS",
        "MONCLER": "RETAILERS",
        "MORROCCANOIL": "PROFESIONALES",
        "NATURA": "FULL",
        "NATURAL PARADISE": "OTROS",
        "NIVEA": "FULL",
        "NOPIKEX": "OTROS",
        "NOVAVENTA FPT": "SISTEMA DE VENTAS",
        "NUDE": "NUDE",
        "OGX": "OTROS",
        "OLAY": "OTROS",
        "OMNILIFE": "SISTEMA DE VENTAS",
        "OTRAS": "OTROS",
        "PREBEL": "OTROS",
        "QVS": "OTROS",
        "SALLY HANSEN": "OTROS",
        "SIN ASIGNAR": "FULL",
        "SOLLA": "FULL",
        "ST. IVES": "OTROS",
        "UBU": "OTROS",
        "UNILEVER": "TOLL",
        "VENTA DIRECTA COSMÉTICOS": "FULL",
        "VITÚ": "OTROS",
        "VITÚ  EXPORTACIÓN": "OTROS",
        "WELLA CONSUMO": "OTROS",
        "WELLA PROFESSIONAL": "PROFESIONALES",
        "YARDLEY": "OTROS",
        "D1": "RETAILERS",
        "CATÁLOGO DE PRODUCTOS": "RETAILERS",
        "WORMSER": "RETAILERS",
        "PROCTER AND GAMBLE": "FULL",
        "DAVINES": "OTROS",
        "LA FABRIL": "OTROS",
        "REVOX": "OTROS",
        "TENDENCIAS AB": "RETAILERS"
    }


def insert_segments() -> Dict[str, str]:
    """
    Retorna un diccionario con el mapeo de marcas QM a segmentaciones.
    """
    # TODO: Implementar el diccionario de segmentaciones según la lógica de negocio
    return {
        "OTROS CLIENTES DO": "DUEÑOS DE CANAL",
        "MARKETING PERSONAL": "DUEÑOS DE CANAL",
        "OMNILIFE": "DUEÑOS DE CANAL",
        "JERONIMO MARTINS": "DUEÑOS DE CANAL",
        "LEONISA": "DUEÑOS DE CANAL",
        "LOCATEL": "DUEÑOS DE CANAL",
        "NOVAVENTA": "DUEÑOS DE CANAL",
        "EL ÉXITO": "DUEÑOS DE CANAL",
        "MILAGROS ENTERPRISE": "DUEÑOS DE CANAL",
        "LA POPULAR": "DUEÑOS DE CANAL",
        "D1": "DUEÑOS DE CANAL",
        "USA": "DUEÑOS DE CANAL",
        "USA": "DUEÑOS DE CANAL",
        "EL EXITO": "DUEÑOS DE CANAL",
        "NOVAVENTA FPT": "DUEÑOS DE CANAL",
        "MILAGROS": "DUEÑOS DE CANAL",
        "WORMSER": "DUEÑOS DE CANAL",
        "TENDENCIAS AB": "DUEÑOS DE CANAL",
        "LA FABRIL": "DUEÑOS DE CANAL",
        "UNILEVER": "EXPERTOS LOCALES",
        "NATURA": "EXPERTOS LOCALES",
        "BIOTECNIK": "EXPERTOS LOCALES",
        "NIVEA": "EXPERTOS LOCALES",
        "BRITO": "EXPERTOS LOCALES",
        "AVON": "EXPERTOS LOCALES",
        "OTROS EXPERTOS LOCALES": "EXPERTOS LOCALES",
        "ALICORP": "EXPERTOS LOCALES",
        "SOLLA": "EXPERTOS LOCALES",
        "ECAR": "EXPERTOS LOCALES",
        "FISA": "EXPERTOS LOCALES",
        "KIMBERLY": "EXPERTOS LOCALES",
        "BELCORP": "EXPERTOS LOCALES",
        "AMWAY": "EXPERTOS LOCALES",
        "PROCTER AND GAMBLE": "EXPERTOS LOCALES",
        "HENKEL": "EXPERTOS LOCALES",
        "DIAL": "EXPERTOS LOCALES",
        "BEIERSDORF": "EXPERTOS LOCALES",
        "OTROS EXPERTOS LOCALES": "EXPERTOS LOCALES",
        "FAMILIA": "EXPERTOS LOCALES",
        "BALANCE": "EXPERTOS LOCALES",
        "MAX FACTOR": "EXPERTOS NO LOCALES",
        "DYCLASS": "EXPERTOS NO LOCALES",
        "WELLA CONSUMO": "EXPERTOS NO LOCALES",
        "BIO OIL": "EXPERTOS NO LOCALES",
        "OGX": "EXPERTOS NO LOCALES",
        "COVER GIRL": "EXPERTOS NO LOCALES",
        "WELLA PROFESSIONAL": "EXPERTOS NO LOCALES",
        "ADIDAS": "EXPERTOS NO LOCALES",
        "ACCESORIOS": "EXPERTOS NO LOCALES",
        "BURTS_BEES": "EXPERTOS NO LOCALES",
        "NOPIKEX": "EXPERTOS NO LOCALES",
        "QVS": "EXPERTOS NO LOCALES",
        "UBU": "EXPERTOS NO LOCALES",
        "ESSENCE": "EXPERTOS NO LOCALES",
        "MORROCCANOIL": "EXPERTOS NO LOCALES",
        "HASK": "EXPERTOS NO LOCALES",
        "HERBAL ESSENCES": "EXPERTOS NO LOCALES",
        "LOVE, BEAUTY AND PLANET": "EXPERTOS NO LOCALES",
        "CATRICE": "EXPERTOS NO LOCALES",
        "NATURAL PARADISE": "EXPERTOS NO LOCALES",
        "OLAY": "EXPERTOS NO LOCALES",
        "MID": "EXPERTOS NO LOCALES",
        "SECRET": "EXPERTOS NO LOCALES",
        "FEBREZE": "EXPERTOS NO LOCALES",
        "TAMPAX": "EXPERTOS NO LOCALES",
        "OFCORSS C.I HERMECO": "EXPERTOS NO LOCALES",
        "CADIVEU": "EXPERTOS NO LOCALES",
        "MAX FACTOR GLOBAL": "EXPERTOS NO LOCALES",
        "SEBASTIAN": "EXPERTOS NO LOCALES",
        "AFFRESH": "EXPERTOS NO LOCALES",
        "COSMETRIX": "EXPERTOS NO LOCALES",
        "INCENTIVOS MAX FACTOR": "EXPERTOS NO LOCALES",
        "OTROS EXPERTOS NO LOCALES": "EXPERTOS NO LOCALES",
        "DAVINES": "EXPERTOS NO LOCALES",
        "P&G": "EXPERTOS NO LOCALES",
        "REVOX": "EXPERTOS NO LOCALES",
        "UTOPICK": "EXPERTOS NO LOCALES",
        "IMPORTADOS PROCTER": "EXPERTOS NO LOCALES",
        "ST. IVES": "EXPERTOS NO LOCALES",
        "ARDEN FOR MEN": "MARCAS PROPIAS",
        "NUDE": "MARCAS PROPIAS",
        "ELIZABETH ARDEN": "MARCAS PROPIAS",
        "YARDLEY": "MARCAS PROPIAS",
        "VITÚ": "MARCAS PROPIAS",
        "PREBEL": "MARCAS PROPIAS",
        "OTRAS MP": "MARCAS PROPIAS",
        "AFM/CFM": "MARCAS PROPIAS",
        "BODY CLEAR": "NO APLICA",
        "OTRAS": "NO APLICA",
        "GILLETTE": "NO APLICA",
        "L&G ASOCIADOS": "NO APLICA",
        "CATÁLOGO DE PRODUCTOS": "NO APLICA",
        "HINODE": "NO APLICA",
        "PFIZER": "NO APLICA",
        "CONTEXPORT DISNEY": "NO APLICA",
        "SYSTEM PROFESSIONAL": "NO APLICA",
        "WELONDA": "NO APLICA",
        "SIN ASIGNAR": "NO APLICA",
        "CATÁLOGO DE PRODUCTOS": "NO APLICA",
    }


def calculate_rango_permanencia(row: pd.Series) -> str:
    """
    Calcula el rango de permanencia basado en las condiciones especificadas.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de permanencia calculado
    """
    lote = row.get("LOTE", None)
    permanencia = row.get("PERMANENCIA", None)
    rango_permanencia = row.get("RANGO DE PERMANENCIA", None)

    if lote == "222222":
        return "1.MENOR DE 90 DIAS"
    elif permanencia == 0 and rango_permanencia == "5.MAYOR O IGUAL A 360 DIAS":
        return "5.ENTRE 360 Y 540 DIAS"
    elif rango_permanencia == "5.MAYOR O IGUAL A 360 DIAS":
        if permanencia < 540:
            return "5.ENTRE 360 Y 540 DIAS"
        elif permanencia < 720:
            return "6.ENTRE 540 Y 720 DIAS"
        else:
            return "7.MAYOR DE 720 DIAS"
    else:
        return rango_permanencia


def calculate_status_cons(row: pd.Series) -> str:
    """
    Calcula el estado de consumo basado en las condiciones de vencimiento, bloqueo y obsolescencia.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Estado calculado (VENCIDO, BLOQUEADO, OBSOLETO, PAV o DISPONIBLE)
    """
    rango_prox_vencer = row.get("RANGO PRÓX.VENCER MM")
    valor_BLOQUEADO = row.get("VALOR BLOQUEADO MM")
    valor_OBSOLETO = row.get("VALOR OBSOLETO")

    if rango_prox_vencer == "VENCIDO":
        return "VENCIDO"
    elif valor_BLOQUEADO != 0:
        return "BLOQUEADO"
    elif valor_OBSOLETO != 0:
        return "OBSOLETO"
    elif rango_prox_vencer in ["1.PAV 3 MESES", "2.PAV 4 A 6 MESES"]:
        return "PAV"
    else:
        return "DISPONIBLE"


def calculate_valor_def(row: pd.Series) -> float:
    """
    Calcula el valor definitivo basado en el estado de consumo.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        float: Valor definitivo calculado
    """
    status_cons = row.get("STATUS CONS")
    valor_BLOQUEADO = row.get("VALOR BLOQUEADO MM")
    valor_total = row.get("VALOR TOTAL MM")

    return valor_BLOQUEADO if status_cons == "BLOQUEADO" else valor_total


def calculate_rango_obsolescencia(row: pd.Series) -> str:
    """
    Calcula el rango de obsolescencia basado en las fechas de entrada y obsolescencia.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de obsolescencia calculado
    """
    status_cons = row.get("STATUS CONS")
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_OBSOLETO = row.get("FECHA OBSOLETO")

    if status_cons != "OBSOLETO" or pd.isna(fecha_entrada) or pd.isna(fecha_OBSOLETO):
        return "FALSO"

    dias_obsolescencia = (fecha_entrada - fecha_OBSOLETO).days

    if dias_obsolescencia <= 90:
        return "1.MENOR DE 90 DIAS"
    elif 90 < dias_obsolescencia <= 180:
        return "2.ENTRE 90 Y 180 DIAS"
    elif 180 < dias_obsolescencia <= 270:
        return "3.ENTRE 180 Y 270 DIAS"
    elif 270 < dias_obsolescencia <= 360:
        return "4.ENTRE 270 Y 360 DIAS"
    elif 360 < dias_obsolescencia <= 540:
        return "5.ENTRE 360 Y 540 DIAS"
    elif 540 < dias_obsolescencia <= 720:
        return "6.ENTRE 540 Y 720 DIAS"
    else:
        return "7.MAYOR DE 720 DIAS"


def calculate_rango_vencido(row: pd.Series) -> str:
    """
    Calcula el rango de vencimiento basado en las fechas de entrada y caducidad.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de vencimiento calculado
    """
    status_cons = row.get("STATUS CONS")
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_caducidad = row.get("FECH, CADUCIDAD/FECH PREF. CONSUMO")

    if status_cons != "VENCIDO" or pd.isna(fecha_entrada) or pd.isna(fecha_caducidad):
        return "FALSO"

    dias_vencido = (fecha_entrada - fecha_caducidad).days

    if dias_vencido <= 90:
        return "1.MENOR DE 90 DIAS"
    elif 90 < dias_vencido <= 180:
        return "2.ENTRE 90 Y 180 DIAS"
    elif 180 < dias_vencido <= 270:
        return "3.ENTRE 180 Y 270 DIAS"
    elif 270 < dias_vencido <= 360:
        return "4.ENTRE 270 Y 360 DIAS"
    elif 360 < dias_vencido <= 540:
        return "5.ENTRE 360 Y 540 DIAS"
    elif 540 < dias_vencido <= 720:
        return "6.ENTRE 540 Y 720 DIAS"
    else:
        return "7.MAYOR DE 720 DIAS"


def calculate_rango_bloqueado(row: pd.Series) -> str:
    """
    Calcula el rango de bloqueo basado en las fechas de entrada y bloqueo.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de bloqueo calculado
    """
    status_cons = row.get("STATUS CONS")
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_bloqueado = row.get("FECHA BLOQUEADO")

    if status_cons != "BLOQUEADO" or pd.isna(fecha_entrada) or pd.isna(fecha_bloqueado):
        return "FALSO"

    dias_vencido = (fecha_entrada - fecha_bloqueado).days

    if dias_vencido <= 90:
        return "1.MENOR DE 90 DIAS"
    elif 90 < dias_vencido <= 180:
        return "2.ENTRE 90 Y 180 DIAS"
    elif 180 < dias_vencido <= 270:
        return "3.ENTRE 180 Y 270 DIAS"
    elif 270 < dias_vencido <= 360:
        return "4.ENTRE 270 Y 360 DIAS"
    elif 360 < dias_vencido <= 540:
        return "5.ENTRE 360 Y 540 DIAS"
    elif 540 < dias_vencido <= 720:
        return "6.ENTRE 540 Y 720 DIAS"
    else:
        return "7.MAYOR A 720 DIAS"


def calculate_tiempo_bloqueo(row: pd.Series) -> str:
    """
    Calcula el tiempo de bloqueo basado en las fechas de entrada y bloqueo.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Tiempo de bloqueo calculado
    """
    fecha_entrada = row.get("FECHA ENTRADA")
    fecha_bloqueado = row.get("FECHA BLOQUEADO")
    
    if pd.isna(fecha_entrada) or pd.isna(fecha_bloqueado):
        return 0

    tiempo_bloqueado = (fecha_entrada - fecha_bloqueado).days
    
    return tiempo_bloqueado    


def calculate_rango_cons(row: pd.Series) -> str:
    """
    Calcula el rango de consumo final basado en el status y los rangos correspondientes.

    Args:
        row: Fila del DataFrame con las columnas necesarias

    Returns:
        str: Rango de consumo calculado
    """
    status = row.get("STATUS CONS")

    if status == "OBSOLETO":
        return row.get("RANGO OBSOLESCENCIA")
    elif status == "VENCIDO":
        return row.get("RANGO VENCIDO 2")
    elif status == "BLOQUEADO":
        return row.get("RANGO BLOQUEADO 2")
    elif status == "PAV":
        return row.get("RANGO DE PERMANENCIA 2")
    else:
        return row.get("RANGO DE PERMANENCIA 2")


def calculate_otros_marcas_factor_and_class(row: pd.Series, lookup_dict: dict) -> tuple[float, str]:
    """
    Calcula factor provisional y clasificación para el resto de marcas
    (excluyendo Avon/Natura), siguiendo la fórmula de Excel proporcionada.
    """
    seg         = row["SEGMENTACION"]
    status      = row["STATUS CONS"]
    tiempo      = row["TIEMPO BLOQUEO"]
    indic       = row["INDICADOR STOCK ESPEC."]
    perm        = row["PERMANENCIA"]
    tipo_mat    = row["TIPO DE MATERIAL (I)"]
    rango_cons  = str(row["RANGO CONS"]).strip()
    cobertura   = str(row["RANGO COBERTURA"]).strip()
    subseg      = str(row.get("SUBSEGMENTACION", "")).strip()
    prox_vencer = str(row.get("RANGO PRÓX.VENCER MM", "")).strip()
    negocio     = row["NEGOCIO INVENTARIOS"]

    # 1) Dueños de canal / Marcas propias / Expertos no locales + Bloqueado corto → 0%, BAJO
    if seg in ["DUEÑOS DE CANAL", "MARCAS PROPIAS", "EXPERTOS NO LOCALES"] \
       and status == "BLOQUEADO" \
       and tiempo <= 30:
        return 0.0, "BAJO"

    # 2) Indicador stock = "K" → 0%, BAJO
    if indic == "K":
        return 0.0, "BAJO"

    # 3) Disponible/PAV + Granel y permanencia corta → 0%, BAJO
    if status in ["DISPONIBLE", "PAV"] \
       and tipo_mat in ["GRANEL", "GRANEL FAB A TERCERO"] \
       and perm <= 30:
        return 0.0, "BAJO"

    # 4) Marca "OTRAS" o (Disponible + cobertura "") → 0%, BAJO
    if row["MARCA DE QM"] == "OTRAS" \
       or (status == "DISPONIBLE" and cobertura == ""):
        return 0.0, "BAJO"

    # 5) Stock = "W" → lookup según status
    if indic == "W":
        if status in ["VENCIDO", "PAV"]:
            key = f"{seg}{status}{row['RANGO DE PERMANENCIA 2']}"
        elif status == "DISPONIBLE":
            key = f"{seg}{cobertura}{row['RANGO DE PERMANENCIA 2']}"
        else:
            key = f"{seg}{status}{rango_cons}"
        return lookup_dict.get(key, (0.0, "BAJO"))

    # 6) Stock = "SIN ASIGNAR" u "O"
    if indic in ["SIN ASIGNAR", "O"]:
        # a) PAV + próxima a vencer 3 o 4-6 meses
        if status == "PAV" and prox_vencer in ["1.PAV 3 MESES", "2.PAV 4 A 6 MESES"]:
            key = f"{seg}{subseg}{cobertura}{row['RANGO DE PERMANENCIA 2']}"
            return lookup_dict.get(key, (0.0, "BAJO"))
        # b) Disponible → lookup principal y fallback
        if status == "DISPONIBLE":
            key = f"{seg}{subseg}{cobertura}{row['RANGO DE PERMANENCIA 2']}"
            if key in lookup_dict:
                return lookup_dict[key]
            alt = f"{seg}{subseg}{status} {rango_cons}"
            return lookup_dict.get(alt, (0.0, "BAJO"))
        # c) Obsoleto/Vencido/Bloqueado
        if status in ["OBSOLETO", "VENCIDO", "BLOQUEADO"]:
            key = f"{seg}{subseg}{status} {rango_cons}"
            return lookup_dict.get(key, (0.0, "BAJO"))

    # 7) Fallback por defecto → 0%, BAJO
    return 0.0, "BAJO"


def calculate_base_riesgo_column(row: pd.Series) -> float:
    """
    Calcula el valor de la columna de base riesgo
    """
    if row["CLAS BASE RIESGO"] == "BAJO":
        return 0.0
    else:
        return row["VALOR DEF"]


def calculate_provision_column(row: pd.Series) -> float:
    """
    Calcula el valor de la columna de provisión
    """
    if row["MARCA DE QM"] == "OTRAS":
        return 0.0
    else:
        return row["VALOR DEF"] * row['FACTOR PROV']


def process_dataframe_otras_marcas(df_otras_marcas: pd.DataFrame,df_matrices_otros_tipos: pd.DataFrame) -> pd.DataFrame:
    """
    Procesa el DataFrame aplicando todas las reglas de negocio en el orden específico requerido.

    Args:
        df: DataFrame con los datos de SAP

    Returns:
        pd.DataFrame: DataFrame procesado con todas las columnas calculadas
    """
    # 1. Añadir columnas formuladas de 'MARCA CONCAT' y 'SEGMENTACION'
    df_otras_marcas["MARCA CONCAT"] = df_otras_marcas["MARCA DE QM"].apply(lambda x: insert_marks().get(x, ""))
    df_otras_marcas["SEGMENTACION"] = df_otras_marcas["MARCA DE QM"].apply(
        lambda x: insert_segments().get(x, "OTRAS")
    )
    df_otras_marcas["SUBSEGMENTACION"] = df_otras_marcas["MARCA DE QM"].apply(
        lambda x: insert_subsegmentacion().get(x, "")
    )

    # 2. Calcular 'RANGO DE PERMANENCIA 2'
    required_columns = {"LOTE", "PERMANENCIA", "RANGO DE PERMANENCIA"}
    if required_columns.issubset(df_otras_marcas.columns):
        df_otras_marcas["RANGO DE PERMANENCIA 2"] = df_otras_marcas.apply(calculate_rango_permanencia, axis=1)
    else:
        print(f"Faltan columnas: {required_columns - set(df_otras_marcas.columns)}")

    # 3. Calcular 'STATUS CONS'
    required_columns = {"RANGO PRÓX.VENCER MM", "VALOR BLOQUEADO MM", "VALOR OBSOLETO"}
    if required_columns.issubset(df_otras_marcas.columns):
        df_otras_marcas["STATUS CONS"] = df_otras_marcas.apply(calculate_status_cons, axis=1)
    else:
        print(f"Faltan columnas: {required_columns - set(df_otras_marcas.columns)}")

    # 4. Calcular 'VALOR DEF'
    required_columns = {"STATUS CONS", "VALOR BLOQUEADO MM", "VALOR TOTAL MM"}
    if required_columns.issubset(df_otras_marcas.columns):
        df_otras_marcas["VALOR DEF"] = df_otras_marcas.apply(calculate_valor_def, axis=1)
    else:
        print(f"Faltan columnas: {required_columns - set(df_otras_marcas.columns)}")

    # 5. Reemplazar valores inválidos
    df_otras_marcas.replace("#", np.nan, inplace=True)

    # 6. Convertir columnas de fecha
    date_columns = [
        "FECHA ENTRADA",
        "FECHA OBSOLETO",
        "FECHA BLOQUEADO",
        "FECH. FABRICACIÓN",
        "CREADO EL",
        "FECH, CADUCIDAD/FECH PREF. CONSUMO",
    ]

    for col in date_columns:
        if col in df_otras_marcas.columns:
            df_otras_marcas[col] = pd.to_datetime(df_otras_marcas[col], format="%d/%m/%Y", errors="coerce")

    # 7. Calcular 'RANGO OBSOLESCENCIA'
    df_otras_marcas["RANGO OBSOLESCENCIA"] = df_otras_marcas.apply(calculate_rango_obsolescencia, axis=1)

    # 8. Calcular 'RANGO VENCIDO 2'
    df_otras_marcas["RANGO VENCIDO 2"] = df_otras_marcas.apply(calculate_rango_vencido, axis=1)

    # 9. Calcular 'RANGO BLOQUEADO 2'
    df_otras_marcas["RANGO BLOQUEADO 2"] = df_otras_marcas.apply(calculate_rango_bloqueado, axis=1)

    # 10. Calcular 'RANGO CONS'
    df_otras_marcas["RANGO CONS"] = df_otras_marcas.apply(calculate_rango_cons, axis=1)
    
    # 11. Calcular 'TIEMPO BLOQUEO'
    df_otras_marcas["TIEMPO BLOQUEO"] = df_otras_marcas.apply(calculate_tiempo_bloqueo, axis=1)

    # Construir lookup_dict **una vez** antes del apply
    lookup_dict = {
        str(r["concatenado"]).strip(): (r["factor_prov"], r["clasificacion"])
        for _, r in df_matrices_otros_tipos.iterrows()
    }

    # Aplicar fila a fila y asignar dos nuevas columnas
    df_otras_marcas[["FACTOR PROV", "CLAS BASE RIESGO"]] = df_otras_marcas.apply(
        lambda row: pd.Series(calculate_otros_marcas_factor_and_class(row, lookup_dict)),
        axis=1
    )

    # 13. Calcular 'BASE RIESGO'
    df_otras_marcas["BASE RIESGO"] = df_otras_marcas.apply(calculate_base_riesgo_column, axis=1)
    
    # 14. Calcular 'PROVISION'
    df_otras_marcas["PROVISION"] = df_otras_marcas.apply(calculate_provision_column, axis=1)
    
    return df_otras_marcas

In [24]:
df_final_otras_marcas= process_dataframe_otras_marcas(df_otras_marcas, df_matrices_otros_tipos)

In [27]:
def combine_final_dataframes(
    df_final_avon_natura: pd.DataFrame,
    df_final_otras_marcas: pd.DataFrame
) -> pd.DataFrame:
    """
    Combina en un único DataFrame los resultados procesados para:
      - Avon/Natura (df_final_avon_natura)
      - Resto de marcas (df_final_otras_marcas)

    Devuelve un nuevo DataFrame con todos los registros y reinicia el índice.
    """
    # Verificar que tengan las mismas columnas
    cols1 = list(df_final_avon_natura.columns)
    cols2 = list(df_final_otras_marcas.columns)
    if cols1 != cols2:
        raise ValueError(
            "Los DataFrames no coinciden en sus columnas.\n"
            f"Avon/Natura: {cols1}\n"
            f"Otras marcas: {cols2}"
        )

    # Concatenar uno encima del otro
    df_final_merge = pd.concat([df_final_avon_natura, df_final_otras_marcas], ignore_index=True)
    
    # Columnas en el orden deseado
    columnas_ordenadas = [
        "NEGOCIO INVENTARIOS", "AÑO NATURAL/MES","TIPO MATERIAL INVENTARIO", "MARCA DE QM", "MATERIAL", 
        "DESCRIPCIÓN", "UNIDAD MEDIDA", "CENTRO", "CODIGO ALMACEN CLIENTE", "INDICADOR STOCK ESPEC.",
        "NÚM.STOCK.ESP.", "LOTE", "CREADO EL", "FECH. FABRICACIÓN", "FECH, CADUCIDAD/FECH PREF. CONSUMO",
        "FECHA BLOQUEADO", "FECHA OBSOLETO", "FECHA ENTRADA", "RANGO OBSOLETO 2", "RANGO COBERTURA",
        "RANGO DE PERMANENCIA", "RANGO BLOQUEADO", "RANGO OBSOLETO", "RANGO VENCIDOS", "PRÓXIMO A VENCER",
        "RANGO PRÓX.VENCER MM", "RANGO PRÓXIMOS A VEN", "TIPO DE MATERIAL (I)", "COSTO UNITARIO REAL",
        "INVENTARIO DISPONIBL", "INVENTARIO NO DISPON", "VALOR OBSOLETO", "VALOR BLOQUEADO MM", "VALOR TOTAL MM", "PERMANENCIA",

        "MARCA CONCAT", "SEGMENTACION", "SUBSEGMENTACION",  
        "RANGO DE PERMANENCIA 2",
        "STATUS CONS", "VALOR DEF", "RANGO OBSOLESCENCIA", "RANGO VENCIDO 2",
        "RANGO BLOQUEADO 2", "RANGO CONS", "TIEMPO BLOQUEO", 
        "FACTOR PROV", "CLAS BASE RIESGO", "BASE RIESGO", "PROVISION" 
    ]
    
    # Renombrar columnas duplicadas automáticamente
    nuevos_nombres = []
    conteo = {}
    for col in df_final_merge.columns:
        if col in conteo:
            conteo[col] += 1
            nuevos_nombres.append(f"{col}_{conteo[col]}")
        else:
            conteo[col] = 0
            nuevos_nombres.append(col)
    df_final_merge.columns = nuevos_nombres
    
    df_final_merge = df_final_merge.reindex(columns=columnas_ordenadas)

    return df_final_merge

In [28]:
df_final_merge = combine_final_dataframes(df_final_avon_natura, df_final_otras_marcas)
export_dataframe_to_excel(df_final_merge)

Archivo Excel creado exitosamente: C:\Users\prac.planeacionfi\OneDrive - Prebel S.A BIC\Escritorio\PRUEBAS BASE RIESGO\Análisis_BaseRiesgo_Final_08-05-2025.xlsx


'C:\\Users\\prac.planeacionfi\\OneDrive - Prebel S.A BIC\\Escritorio\\PRUEBAS BASE RIESGO\\Análisis_BaseRiesgo_Final_08-05-2025.xlsx'